# Stock purchase recommendations with Machine Learning

In [317]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm # progress bar

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [318]:
pd.set_option('display.max_columns', 1500)

In [319]:
# read in the Quandl.com API key - saved in separate file to keep it private

filepath = 'confidential-API-key.txt'  
with open(filepath) as fp:  
   my_Quandl_API_key = fp.readline()

## Load stock data from Quandl

In [320]:
def load_stock_data(ticker, start_date, end_date, Quandl_API_key=my_Quandl_API_key):
    '''
    Downloads stock data from Quandl, drop some columns, resort datafram, and return
    Inputs:
        ticker - a stock ticker symbol (needs to be valid - no error checking implemented)
        start_date - first date of stock prices
        end_date - last date of stock prices
        Quandl_API_key - string with valid API key for Quandl.com data queries
    Outputs:
        stock_data - DataFrame with stock price data, sorted in ascending date order
    '''
    
    # download data from Quandl with Pandas Datareader
    stock_data = web.DataReader(name=symbol, data_source='quandl', start=start_date, end=end_date, access_key=Quandl_API_key)
    
    # need ascending index for the TA-lib indicators to work properly
    stock_data.sort_index(inplace=True)
    
    # keep only the columns with adjusted data to eliminate any issues due to stock splits
    stock_data = stock_data[['AdjVolume', 'AdjOpen', 'AdjHigh', 'AdjLow', 'AdjClose']]
    
    # DataFram has a second level column index with the stock ticker - not needed so drop it
    stock_data.columns = stock_data.columns.droplevel(1)
    
    # add row index (highest number is most recent date) - this will be used to later restack the rows for feature matrix
    stock_data['row_index'] = range(0, stock_data.shape[0])
    
    return stock_data    

In [321]:
# test loading stock data:

start = '2013-01-01'
end = '2019-03-30'
symbol = ['AAPL']

df = load_stock_data(symbol, start, end)

df.head()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index
Date,,,,,,
2013-01-02,140129500.0,71.816894,71.969911,70.236149,71.195748,0
2013-01-03,88241300.0,71.046621,71.278740,70.154453,70.296565,1
2013-01-04,148583400.0,69.631213,69.847109,68.187094,68.338996,2
2013-01-07,121039100.0,67.690619,68.637250,66.808825,67.937002,3
2013-01-08,114676800.0,68.625579,68.973109,67.593362,68.119845,4


In [322]:
df.shape

(1316, 6)

## Define Target column: profitability of trade

The goal is the define a column "setup_for_profitable_trade" based on a trading strategy. The data will be analyzed for this trade setup:

Run the analysis in the evening after market close. That day's row of market data will then be either deemed as profitable or not for a trade to be initiated and closed in the following days. This setup of the data ensures no lookahead is happening.

## Trading Strategy:

1. After market close on day N decide if buying stock at market Open on day N+1
2. Open position: submit market order for day N+1 prior to market open
3. Close position: submit sell order for market open for day N+10

Threshold to decide to buy the stock: expected profit from open to close: 2.5% profit

Store in row N if this trade was deemed profitable.

In [323]:
def add_flag_for_profitable_trade_setups(df, delete_interim_calculation_cols=True):
    '''
    Adds column to dataframe that identifies profitable trading setup
    Inputs:
        df - dataframe with stock data
        delete_interim_calculation_cols - if false: keep the interim calculations - good for debugging
    Outputs:
        df - dataframe with additional columns
    
    '''
    
    # trade strategy: after market close on day N, set a buy at market open on day N+1 and sell at market close on day N+1
    # for expected gain of at least 2.5%
    profitability_threshold = 0.025

    # use helper columns to calculate profit
    df['strategy_open_price'] = df['AdjOpen'].shift(-1) # AdjOpen from day N+1
    df['strategy_close_price'] = df['AdjOpen'].shift(-10) # AdjOpen from day N+10
    df['strategy_profit_dollars'] = df['strategy_close_price'] - df['strategy_open_price']
    df['strategy_profitability'] = df['strategy_profit_dollars'] / df['strategy_open_price']

    # use categorical field to encode "setup_for_succesful_trade": 1=yes, 0=no
    df['setup_for_profitable_trade'] = df['strategy_profitability'] >= profitability_threshold
    
    if delete_interim_calculation_cols:
        df = df.drop(columns=['strategy_open_price', 'strategy_close_price', 'strategy_profit_dollars', 'strategy_profitability'])
    
    # drop any rows that have NaNs in them. especially the last few rows will have NaNs because the profitability calc
    # looks into the future beyond the last row. these rows need to get deleted - the profitability is not defined there
    df = df.loc[df.notnull().all(axis=1), :]
    
    return df

In [324]:
# testing: show calculations:
df = add_flag_for_profitable_trade_setups(df, False)
df.tail(10)

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,strategy_open_price,strategy_close_price,strategy_profit_dollars,strategy_profitability,setup_for_profitable_trade
Date,,,,,,,,,,,
2018-02-28,33604574.0,179.26,180.615,178.05,178.12,1296,178.54,180.32,1.78,0.009970,False
2018-03-01,48801970.0,178.54,179.775,172.66,175.00,1297,172.80,178.50,5.70,0.032986,True
2018-03-02,38453950.0,172.80,176.300,172.45,176.21,1298,175.21,178.65,3.44,0.019634,False
2018-03-05,28401366.0,175.21,177.740,174.52,176.82,1299,177.91,177.32,-0.59,-0.003316,False
2018-03-06,23788506.0,177.91,178.250,176.13,176.67,1300,174.94,175.24,0.30,0.001715,False
2018-03-07,31703462.0,174.94,175.850,174.27,175.03,1301,175.48,175.04,-0.44,-0.002507,False
2018-03-08,23163767.0,175.48,177.120,175.07,176.94,1302,177.96,170.00,-7.96,-0.044729,False
2018-03-09,31385134.0,177.96,180.000,177.39,179.98,1303,180.29,168.39,-11.90,-0.066005,False
2018-03-12,32055405.0,180.29,182.390,180.21,181.72,1304,182.59,168.07,-14.52,-0.079522,False


In [325]:
# without verbose results:
df = add_flag_for_profitable_trade_setups(df)
df.tail(10)

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,setup_for_profitable_trade
Date,,,,,,,
2018-02-28,33604574.0,179.26,180.615,178.05,178.12,1296,False
2018-03-01,48801970.0,178.54,179.775,172.66,175.00,1297,False
2018-03-02,38453950.0,172.80,176.300,172.45,176.21,1298,False
2018-03-05,28401366.0,175.21,177.740,174.52,176.82,1299,False
2018-03-06,23788506.0,177.91,178.250,176.13,176.67,1300,False
2018-03-07,31703462.0,174.94,175.850,174.27,175.03,1301,False
2018-03-08,23163767.0,175.48,177.120,175.07,176.94,1302,False
2018-03-09,31385134.0,177.96,180.000,177.39,179.98,1303,False
2018-03-12,32055405.0,180.29,182.390,180.21,181.72,1304,False


In [326]:
df['setup_for_profitable_trade'].sum()

447

## Feature Engineering

In [327]:
#split df into features and target (column setup_for_profitable_trade)
df_X_base_data = df.drop(columns=['setup_for_profitable_trade'])

### Feature Engineering - Add Technical Analysis Indicators

In [328]:
def add_TALib_indicator(df, attribute, indicator_func, *args):
    '''
    Adds a column to a dataframe:
        column name is the name of the technical indicator as specified by indicator_func
        column content is the function calculated on the attribute column
    Example: add_TALib_indicator(df, 'AdjClose', ta.RSI, 14) creates a new column called RSI with 
             the 14 day RSI of the values of the column 'AdjClose'
    Inputs:
        df - dataframe - needs to be sorted in date ascending order
        attribute - column name to be used in TA-Lib calculation
        indicator_func - name of a TA-Lib function
        *args - optional parameters for indicator_func
        
    Oupputs:
        df - datarame with new column added
        func_name - name of the new colunm
    
    '''
    # get the name of the indicator from TA-Lib
    func_name = attribute + indicator_func.__name__ + str(*args)
    
    # add new column, calculated based on attribute column
    df.loc[:, func_name] = indicator_func(df.loc[:, attribute].values, *args)
    
    return df, func_name

In [329]:
def add_comparison_cols_for_indicator(df, base_col_name, indicator_col_name, delete_indicator_col=True):
    '''
    adds columns that compare indicator_col to base_col: ratio, crossover, above/below
    Inputs:
        df - dataframe
        base_col_name - name of column that the indicator will get compared to
        indicator_col_name - name of column that has indicator values
        delete_base_col - yes/no on if to keep the base col or not
    Output:
        df - modified df with added & removed columns
    '''
   
    # indicator to base column ratio:
    df.loc[:, indicator_col_name + '_to_' + base_col_name + '_ratio'] = df.loc[:, indicator_col_name] / df.loc[:, base_col_name]
    
    # base col above indicator:
    base_above_indicator_col_name = base_col_name + '_above_' + indicator_col_name
    df.loc[:, base_above_indicator_col_name] = df.loc[:, indicator_col_name] < df.loc[:, base_col_name]
    
    # did base cross indicator
    base_crossed_indicator_col_name = base_col_name + '_crossed_' + indicator_col_name
    df.loc[:, base_crossed_indicator_col_name] = df.loc[:, base_above_indicator_col_name] != df.loc[:, base_above_indicator_col_name].shift(1)
    
    if delete_indicator_col:
        df = df.drop(columns=indicator_col_name)
    
    return df

In [330]:
df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.RSI, 14)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=False)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 10)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=False)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 50)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=False)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 200)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=False)


df_X_base_data.tail()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI14,AdjCloseRSI14_to_AdjClose_ratio,AdjClose_above_AdjCloseRSI14,AdjClose_crossed_AdjCloseRSI14,AdjCloseSMA10,AdjCloseSMA10_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA10,AdjClose_crossed_AdjCloseSMA10,AdjCloseSMA50,AdjCloseSMA50_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA50,AdjClose_crossed_AdjCloseSMA50,AdjCloseSMA200,AdjCloseSMA200_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA200,AdjClose_crossed_AdjCloseSMA200
Date,,,,,,,,,,,,,,,,,,,,,,
2018-03-07,31703462.0,174.94,175.85,174.27,175.03,1301,55.278233,0.315821,True,False,176.3365,1.007464,False,True,171.5245,0.979972,True,False,161.444627,0.922383,True,False
2018-03-08,23163767.0,175.48,177.12,175.07,176.94,1302,58.558368,0.330950,True,False,176.7705,0.999042,True,True,171.5631,0.969612,True,False,161.569709,0.913133,True,False
2018-03-09,31385134.0,177.96,180.00,177.39,179.98,1303,63.186492,0.351075,True,False,177.2130,0.984626,True,False,171.7513,0.954280,True,False,161.707900,0.898477,True,False
2018-03-12,32055405.0,180.29,182.39,180.21,181.72,1304,65.557450,0.360761,True,False,177.4880,0.976711,True,False,171.9737,0.946366,True,False,161.849661,0.890654,True,False
2018-03-13,31168404.0,182.59,183.50,179.24,179.97,1305,61.282541,0.340515,True,False,177.6460,0.987087,True,False,172.1515,0.956557,True,False,161.983619,0.900059,True,False


In [331]:
df_X_base_data.head(16)
#confirms NaN for RSI on top for first 14 dates since it is a 14 day RSI

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI14,AdjCloseRSI14_to_AdjClose_ratio,AdjClose_above_AdjCloseRSI14,AdjClose_crossed_AdjCloseRSI14,AdjCloseSMA10,AdjCloseSMA10_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA10,AdjClose_crossed_AdjCloseSMA10,AdjCloseSMA50,AdjCloseSMA50_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA50,AdjClose_crossed_AdjCloseSMA50,AdjCloseSMA200,AdjCloseSMA200_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA200,AdjClose_crossed_AdjCloseSMA200
Date,,,,,,,,,,,,,,,,,,,,,,
2013-01-02,140129500.0,71.816894,71.969911,70.236149,71.195748,0,NaN,NaN,False,True,NaN,NaN,False,True,NaN,NaN,False,True,NaN,NaN,False,True
2013-01-03,88241300.0,71.046621,71.278740,70.154453,70.296565,1,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False
2013-01-04,148583400.0,69.631213,69.847109,68.187094,68.338996,2,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False
2013-01-07,121039100.0,67.690619,68.637250,66.808825,67.937002,3,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False
2013-01-08,114676800.0,68.625579,68.973109,67.593362,68.119845,4,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False
2013-01-09,101901100.0,67.755456,68.080345,66.911269,67.055209,5,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False
2013-01-10,150286500.0,68.539993,68.562038,66.850321,67.886429,6,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False
2013-01-11,87626700.0,67.560943,68.121141,67.304186,67.470170,7,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False
2013-01-14,183551900.0,65.185288,65.810324,64.644541,65.064689,8,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False,NaN,NaN,False,False


### Feature Engineering - Changing from actual values to percentage changes

In [332]:
def feat_eng_changes_values_to_change(df, cols_set_vals_to_change, delete_original_cols=True):
    '''
    Instead of the actual values in some columns, we care about the change from one day to the next.
    This function calculates that change for the given columns and then either keeps or drops (default) the origianl columns
    Input:
        df - a dataframe
        cols_set_vals_to_change - names of columns to work on.
        delete_original_cols - keep or delete original columns
    Output:
        df - dataframe with new columns added. the value in row N is now the change from row N-1 to row N (instead of the actual values)
    '''    

    # calculate the change from row N-1 to row N
    df_chg_cols = (df[cols_set_vals_to_change] / df[cols_set_vals_to_change].shift(1) - 1)

    # add suffix to the column names
    df_chg_cols = df_chg_cols.add_suffix('_chg')

    # join the data onto the original data fram
    df = df.join(df_chg_cols)

    if delete_original_cols:
        # drop the original columns
        df = df.drop(columns=cols_set_vals_to_change)
        
    return df

cols_set_vals_to_change = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']
df_X_base_data = feat_eng_changes_values_to_change(df_X_base_data, cols_set_vals_to_change, delete_original_cols=False)

df_X_base_data.tail()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI14,AdjCloseRSI14_to_AdjClose_ratio,AdjClose_above_AdjCloseRSI14,AdjClose_crossed_AdjCloseRSI14,AdjCloseSMA10,AdjCloseSMA10_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA10,AdjClose_crossed_AdjCloseSMA10,AdjCloseSMA50,AdjCloseSMA50_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA50,AdjClose_crossed_AdjCloseSMA50,AdjCloseSMA200,AdjCloseSMA200_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA200,AdjClose_crossed_AdjCloseSMA200,AdjVolume_chg,AdjOpen_chg,AdjLow_chg,AdjHigh_chg,AdjClose_chg
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-03-07,31703462.0,174.94,175.85,174.27,175.03,1301,55.278233,0.315821,True,False,176.3365,1.007464,False,True,171.5245,0.979972,True,False,161.444627,0.922383,True,False,0.332722,-0.016694,-0.010560,-0.013464,-0.009283
2018-03-08,23163767.0,175.48,177.12,175.07,176.94,1302,58.558368,0.330950,True,False,176.7705,0.999042,True,True,171.5631,0.969612,True,False,161.569709,0.913133,True,False,-0.269362,0.003087,0.004591,0.007222,0.010912
2018-03-09,31385134.0,177.96,180.00,177.39,179.98,1303,63.186492,0.351075,True,False,177.2130,0.984626,True,False,171.7513,0.954280,True,False,161.707900,0.898477,True,False,0.354924,0.014133,0.013252,0.016260,0.017181
2018-03-12,32055405.0,180.29,182.39,180.21,181.72,1304,65.557450,0.360761,True,False,177.4880,0.976711,True,False,171.9737,0.946366,True,False,161.849661,0.890654,True,False,0.021356,0.013093,0.015897,0.013278,0.009668
2018-03-13,31168404.0,182.59,183.50,179.24,179.97,1305,61.282541,0.340515,True,False,177.6460,0.987087,True,False,172.1515,0.956557,True,False,161.983619,0.900059,True,False,-0.027671,0.012757,-0.005383,0.006086,-0.009630


### Feature Engineering - Reshaping the prior day data into ML-ready features matrix
#### --- build pieces for the dataframe section unpivoting ---
Goal: pull out n_features rows, flatten, and build meaningful column names that indicate how many days' back the data is from

In [333]:
# parameter for how many days of history to include in feature list
n_days_features = 5

# total lenght of df
n_data_points = df.shape[0]

In [334]:
i = 505
# pull out n_days_features of rows from current position
df_extract = df_X_base_data.iloc[i-n_days_features:i, :].copy()

# change the index the be "days into the past" - eg current day is 0, prior day is -1, ...
df_extract.loc[:, 'row_index'] = range(-n_days_features+1, 1)

# make this the new index
df_extract.set_index('row_index', inplace=True)

df_extract

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjCloseRSI14,AdjCloseRSI14_to_AdjClose_ratio,AdjClose_above_AdjCloseRSI14,AdjClose_crossed_AdjCloseRSI14,AdjCloseSMA10,AdjCloseSMA10_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA10,AdjClose_crossed_AdjCloseSMA10,AdjCloseSMA50,AdjCloseSMA50_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA50,AdjClose_crossed_AdjCloseSMA50,AdjCloseSMA200,AdjCloseSMA200_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA200,AdjClose_crossed_AdjCloseSMA200,AdjVolume_chg,AdjOpen_chg,AdjLow_chg,AdjHigh_chg,AdjClose_chg
row_index,,,,,,,,,,,,,,,,,,,,,,,,,,
-4,33720951.0,106.495469,108.794479,106.409968,108.290977,55.633496,0.513741,True,False,105.452364,0.973787,True,False,104.980304,0.969428,True,False,90.551157,0.836184,True,False,1.328858,-0.004264,0.000000,0.016059,0.017677
-3,27598920.0,108.100976,109.031980,108.015476,108.214976,55.382828,0.511785,True,False,105.849466,0.978141,True,False,105.323529,0.973281,True,False,90.741442,0.838529,True,False,-0.181550,0.015076,0.015088,0.002183,-0.000702
-2,29881477.0,107.958475,108.224476,106.504969,106.894471,51.076636,0.477823,True,False,106.257493,0.994041,True,False,105.613670,0.988018,True,False,90.923753,0.850594,True,False,0.082705,-0.001318,-0.013984,-0.007406,-0.012203
-1,41403351.0,107.179472,107.473973,104.699961,104.861462,45.244023,0.431465,True,False,106.602819,1.016606,False,True,105.823611,1.009175,False,True,91.092783,0.868696,True,False,0.385586,-0.007216,-0.016948,-0.006935,-0.019019
0,53204626.0,105.820966,105.868466,101.982949,103.863957,42.669377,0.410820,True,False,106.595219,1.026297,False,False,105.962334,1.020203,False,False,91.256920,0.878620,True,False,0.285032,-0.012675,-0.025950,-0.014939,-0.009513


In [335]:
# normalize columns from dollars to "1" - turns the columns into ratios compared to day N (index 0)

cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']

df_extract.loc[:, cols_to_normalize_to_1_for_day_0] = df_extract.loc[:, cols_to_normalize_to_1_for_day_0] / df_extract.loc[0, cols_to_normalize_to_1_for_day_0]

df_extract

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjCloseRSI14,AdjCloseRSI14_to_AdjClose_ratio,AdjClose_above_AdjCloseRSI14,AdjClose_crossed_AdjCloseRSI14,AdjCloseSMA10,AdjCloseSMA10_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA10,AdjClose_crossed_AdjCloseSMA10,AdjCloseSMA50,AdjCloseSMA50_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA50,AdjClose_crossed_AdjCloseSMA50,AdjCloseSMA200,AdjCloseSMA200_to_AdjClose_ratio,AdjClose_above_AdjCloseSMA200,AdjClose_crossed_AdjCloseSMA200,AdjVolume_chg,AdjOpen_chg,AdjLow_chg,AdjHigh_chg,AdjClose_chg
row_index,,,,,,,,,,,,,,,,,,,,,,,,,,
-4,0.633797,1.006374,1.027638,1.043409,1.042623,55.633496,0.513741,True,False,105.452364,0.973787,True,False,104.980304,0.969428,True,False,90.551157,0.836184,True,False,1.328858,-0.004264,0.000000,0.016059,0.017677
-3,0.518732,1.021546,1.029882,1.059152,1.041892,55.382828,0.511785,True,False,105.849466,0.978141,True,False,105.323529,0.973281,True,False,90.741442,0.838529,True,False,-0.181550,0.015076,0.015088,0.002183,-0.000702
-2,0.561633,1.020199,1.022254,1.044341,1.029178,51.076636,0.477823,True,False,106.257493,0.994041,True,False,105.613670,0.988018,True,False,90.923753,0.850594,True,False,0.082705,-0.001318,-0.013984,-0.007406,-0.012203
-1,0.778191,1.012838,1.015165,1.026642,1.009604,45.244023,0.431465,True,False,106.602819,1.016606,False,True,105.823611,1.009175,False,True,91.092783,0.868696,True,False,0.385586,-0.007216,-0.016948,-0.006935,-0.019019
0,1.000000,1.000000,1.000000,1.000000,1.000000,42.669377,0.410820,True,False,106.595219,1.026297,False,False,105.962334,1.020203,False,False,91.256920,0.878620,True,False,0.285032,-0.012675,-0.025950,-0.014939,-0.009513


In [336]:
# unstack and make it tall (ie unpivot)
df_extract = df_extract.unstack().reset_index()
df_extract

,Attributes,row_index,0
0,AdjVolume,-4,0.633797
1,AdjVolume,-3,0.518732
2,AdjVolume,-2,0.561633
3,AdjVolume,-1,0.778191
4,AdjVolume,0,1
5,AdjOpen,-4,1.00637
6,AdjOpen,-3,1.02155
7,AdjOpen,-2,1.0202
8,AdjOpen,-1,1.01284
9,AdjOpen,0,1


In [337]:
# create new column with combined field names of attribute and index
# eg: AdjClose-1 for the adjusted close of day N-1 or AdjHigh-4 for the adjusted High of day N-4
df_extract['Attribute-index'] = df_extract['Attributes'] + df_extract['row_index'].apply(str)
# then drop Attributes and row_index columns since they are not needed anymore
df_extract.drop(columns=['Attributes', 'row_index'], inplace=True)
df_extract

,0,Attribute-index
0,0.633797,AdjVolume-4
1,0.518732,AdjVolume-3
2,0.561633,AdjVolume-2
3,0.778191,AdjVolume-1
4,1,AdjVolume0
5,1.00637,AdjOpen-4
6,1.02155,AdjOpen-3
7,1.0202,AdjOpen-2
8,1.01284,AdjOpen-1
9,1,AdjOpen0


In [338]:
# set index one and transpose
target_row = df_extract.set_index('Attribute-index').T
# we now have one row of data that represents the prior n_feature_days worth of data:
target_row

Attribute-index,AdjVolume-4,AdjVolume-3,AdjVolume-2,AdjVolume-1,AdjVolume0,AdjOpen-4,AdjOpen-3,AdjOpen-2,AdjOpen-1,AdjOpen0,AdjHigh-4,AdjHigh-3,AdjHigh-2,AdjHigh-1,AdjHigh0,AdjLow-4,AdjLow-3,AdjLow-2,AdjLow-1,AdjLow0,AdjClose-4,AdjClose-3,AdjClose-2,AdjClose-1,AdjClose0,AdjCloseRSI14-4,AdjCloseRSI14-3,AdjCloseRSI14-2,AdjCloseRSI14-1,AdjCloseRSI140,AdjCloseRSI14_to_AdjClose_ratio-4,AdjCloseRSI14_to_AdjClose_ratio-3,AdjCloseRSI14_to_AdjClose_ratio-2,AdjCloseRSI14_to_AdjClose_ratio-1,AdjCloseRSI14_to_AdjClose_ratio0,AdjClose_above_AdjCloseRSI14-4,AdjClose_above_AdjCloseRSI14-3,AdjClose_above_AdjCloseRSI14-2,AdjClose_above_AdjCloseRSI14-1,AdjClose_above_AdjCloseRSI140,AdjClose_crossed_AdjCloseRSI14-4,AdjClose_crossed_AdjCloseRSI14-3,AdjClose_crossed_AdjCloseRSI14-2,AdjClose_crossed_AdjCloseRSI14-1,AdjClose_crossed_AdjCloseRSI140,AdjCloseSMA10-4,AdjCloseSMA10-3,AdjCloseSMA10-2,AdjCloseSMA10-1,AdjCloseSMA100,AdjCloseSMA10_to_AdjClose_ratio-4,AdjCloseSMA10_to_AdjClose_ratio-3,AdjCloseSMA10_to_AdjClose_ratio-2,AdjCloseSMA10_to_AdjClose_ratio-1,AdjCloseSMA10_to_AdjClose_ratio0,AdjClose_above_AdjCloseSMA10-4,AdjClose_above_AdjCloseSMA10-3,AdjClose_above_AdjCloseSMA10-2,AdjClose_above_AdjCloseSMA10-1,AdjClose_above_AdjCloseSMA100,AdjClose_crossed_AdjCloseSMA10-4,AdjClose_crossed_AdjCloseSMA10-3,AdjClose_crossed_AdjCloseSMA10-2,AdjClose_crossed_AdjCloseSMA10-1,AdjClose_crossed_AdjCloseSMA100,AdjCloseSMA50-4,AdjCloseSMA50-3,AdjCloseSMA50-2,AdjCloseSMA50-1,AdjCloseSMA500,AdjCloseSMA50_to_AdjClose_ratio-4,AdjCloseSMA50_to_AdjClose_ratio-3,AdjCloseSMA50_to_AdjClose_ratio-2,AdjCloseSMA50_to_AdjClose_ratio-1,AdjCloseSMA50_to_AdjClose_ratio0,AdjClose_above_AdjCloseSMA50-4,AdjClose_above_AdjCloseSMA50-3,AdjClose_above_AdjCloseSMA50-2,AdjClose_above_AdjCloseSMA50-1,AdjClose_above_AdjCloseSMA500,AdjClose_crossed_AdjCloseSMA50-4,AdjClose_crossed_AdjCloseSMA50-3,AdjClose_crossed_AdjCloseSMA50-2,AdjClose_crossed_AdjCloseSMA50-1,AdjClose_crossed_AdjCloseSMA500,AdjCloseSMA200-4,AdjCloseSMA200-3,AdjCloseSMA200-2,AdjCloseSMA200-1,AdjCloseSMA2000,AdjCloseSMA200_to_AdjClose_ratio-4,AdjCloseSMA200_to_AdjClose_ratio-3,AdjCloseSMA200_to_AdjClose_ratio-2,AdjCloseSMA200_to_AdjClose_ratio-1,AdjCloseSMA200_to_AdjClose_ratio0,AdjClose_above_AdjCloseSMA200-4,AdjClose_above_AdjCloseSMA200-3,AdjClose_above_AdjCloseSMA200-2,AdjClose_above_AdjCloseSMA200-1,AdjClose_above_AdjCloseSMA2000,AdjClose_crossed_AdjCloseSMA200-4,AdjClose_crossed_AdjCloseSMA200-3,AdjClose_crossed_AdjCloseSMA200-2,AdjClose_crossed_AdjCloseSMA200-1,AdjClose_crossed_AdjCloseSMA2000,AdjVolume_chg-4,AdjVolume_chg-3,AdjVolume_chg-2,AdjVolume_chg-1,AdjVolume_chg0,AdjOpen_chg-4,AdjOpen_chg-3,AdjOpen_chg-2,AdjOpen_chg-1,AdjOpen_chg0,AdjLow_chg-4,AdjLow_chg-3,AdjLow_chg-2,AdjLow_chg-1,AdjLow_chg0,AdjHigh_chg-4,AdjHigh_chg-3,AdjHigh_chg-2,AdjHigh_chg-1,AdjHigh_chg0,AdjClose_chg-4,AdjClose_chg-3,AdjClose_chg-2,AdjClose_chg-1,AdjClose_chg0
0,0.633797,0.518732,0.561633,0.778191,1,1.00637,1.02155,1.0202,1.01284,1,1.02764,1.02988,1.02225,1.01517,1,1.04341,1.05915,1.04434,1.02664,1,1.04262,1.04189,1.02918,1.0096,1,55.6335,55.3828,51.0766,45.244,42.6694,0.513741,0.511785,0.477823,0.431465,0.41082,True,True,True,True,True,False,False,False,False,False,105.452,105.849,106.257,106.603,106.595,0.973787,0.978141,0.994041,1.01661,1.0263,True,True,True,False,False,False,False,False,True,False,104.98,105.324,105.614,105.824,105.962,0.969428,0.973281,0.988018,1.00918,1.0202,True,True,True,False,False,False,False,False,True,False,90.5512,90.7414,90.9238,91.0928,91.2569,0.836184,0.838529,0.850594,0.868696,0.87862,True,True,True,True,True,False,False,False,False,False,1.32886,-0.18155,0.0827046,0.385586,0.285032,-0.00426363,0.0150758,-0.00131822,-0.00721577,-0.0126751,0,0.0150879,-0.0139842,-0.0169476,-0.0259505,0.0160589,0.00218302,-0.00740612,-0.00693469,-0.0149386,0.017677,-0.000701816,-0.0122026,-0.0190188,-0.00951259


In [339]:
df_X_base_data.index.to_list()[504]

Timestamp('2015-01-02 00:00:00')

In [340]:
target_row['Index'] = df_X_base_data.index.to_list()[504]
target_row = target_row.set_index('Index')
target_row

Attribute-index,AdjVolume-4,AdjVolume-3,AdjVolume-2,AdjVolume-1,AdjVolume0,AdjOpen-4,AdjOpen-3,AdjOpen-2,AdjOpen-1,AdjOpen0,AdjHigh-4,AdjHigh-3,AdjHigh-2,AdjHigh-1,AdjHigh0,AdjLow-4,AdjLow-3,AdjLow-2,AdjLow-1,AdjLow0,AdjClose-4,AdjClose-3,AdjClose-2,AdjClose-1,AdjClose0,AdjCloseRSI14-4,AdjCloseRSI14-3,AdjCloseRSI14-2,AdjCloseRSI14-1,AdjCloseRSI140,AdjCloseRSI14_to_AdjClose_ratio-4,AdjCloseRSI14_to_AdjClose_ratio-3,AdjCloseRSI14_to_AdjClose_ratio-2,AdjCloseRSI14_to_AdjClose_ratio-1,AdjCloseRSI14_to_AdjClose_ratio0,AdjClose_above_AdjCloseRSI14-4,AdjClose_above_AdjCloseRSI14-3,AdjClose_above_AdjCloseRSI14-2,AdjClose_above_AdjCloseRSI14-1,AdjClose_above_AdjCloseRSI140,AdjClose_crossed_AdjCloseRSI14-4,AdjClose_crossed_AdjCloseRSI14-3,AdjClose_crossed_AdjCloseRSI14-2,AdjClose_crossed_AdjCloseRSI14-1,AdjClose_crossed_AdjCloseRSI140,AdjCloseSMA10-4,AdjCloseSMA10-3,AdjCloseSMA10-2,AdjCloseSMA10-1,AdjCloseSMA100,AdjCloseSMA10_to_AdjClose_ratio-4,AdjCloseSMA10_to_AdjClose_ratio-3,AdjCloseSMA10_to_AdjClose_ratio-2,AdjCloseSMA10_to_AdjClose_ratio-1,AdjCloseSMA10_to_AdjClose_ratio0,AdjClose_above_AdjCloseSMA10-4,AdjClose_above_AdjCloseSMA10-3,AdjClose_above_AdjCloseSMA10-2,AdjClose_above_AdjCloseSMA10-1,AdjClose_above_AdjCloseSMA100,AdjClose_crossed_AdjCloseSMA10-4,AdjClose_crossed_AdjCloseSMA10-3,AdjClose_crossed_AdjCloseSMA10-2,AdjClose_crossed_AdjCloseSMA10-1,AdjClose_crossed_AdjCloseSMA100,AdjCloseSMA50-4,AdjCloseSMA50-3,AdjCloseSMA50-2,AdjCloseSMA50-1,AdjCloseSMA500,AdjCloseSMA50_to_AdjClose_ratio-4,AdjCloseSMA50_to_AdjClose_ratio-3,AdjCloseSMA50_to_AdjClose_ratio-2,AdjCloseSMA50_to_AdjClose_ratio-1,AdjCloseSMA50_to_AdjClose_ratio0,AdjClose_above_AdjCloseSMA50-4,AdjClose_above_AdjCloseSMA50-3,AdjClose_above_AdjCloseSMA50-2,AdjClose_above_AdjCloseSMA50-1,AdjClose_above_AdjCloseSMA500,AdjClose_crossed_AdjCloseSMA50-4,AdjClose_crossed_AdjCloseSMA50-3,AdjClose_crossed_AdjCloseSMA50-2,AdjClose_crossed_AdjCloseSMA50-1,AdjClose_crossed_AdjCloseSMA500,AdjCloseSMA200-4,AdjCloseSMA200-3,AdjCloseSMA200-2,AdjCloseSMA200-1,AdjCloseSMA2000,AdjCloseSMA200_to_AdjClose_ratio-4,AdjCloseSMA200_to_AdjClose_ratio-3,AdjCloseSMA200_to_AdjClose_ratio-2,AdjCloseSMA200_to_AdjClose_ratio-1,AdjCloseSMA200_to_AdjClose_ratio0,AdjClose_above_AdjCloseSMA200-4,AdjClose_above_AdjCloseSMA200-3,AdjClose_above_AdjCloseSMA200-2,AdjClose_above_AdjCloseSMA200-1,AdjClose_above_AdjCloseSMA2000,AdjClose_crossed_AdjCloseSMA200-4,AdjClose_crossed_AdjCloseSMA200-3,AdjClose_crossed_AdjCloseSMA200-2,AdjClose_crossed_AdjCloseSMA200-1,AdjClose_crossed_AdjCloseSMA2000,AdjVolume_chg-4,AdjVolume_chg-3,AdjVolume_chg-2,AdjVolume_chg-1,AdjVolume_chg0,AdjOpen_chg-4,AdjOpen_chg-3,AdjOpen_chg-2,AdjOpen_chg-1,AdjOpen_chg0,AdjLow_chg-4,AdjLow_chg-3,AdjLow_chg-2,AdjLow_chg-1,AdjLow_chg0,AdjHigh_chg-4,AdjHigh_chg-3,AdjHigh_chg-2,AdjHigh_chg-1,AdjHigh_chg0,AdjClose_chg-4,AdjClose_chg-3,AdjClose_chg-2,AdjClose_chg-1,AdjClose_chg0
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.633797,0.518732,0.561633,0.778191,1,1.00637,1.02155,1.0202,1.01284,1,1.02764,1.02988,1.02225,1.01517,1,1.04341,1.05915,1.04434,1.02664,1,1.04262,1.04189,1.02918,1.0096,1,55.6335,55.3828,51.0766,45.244,42.6694,0.513741,0.511785,0.477823,0.431465,0.41082,True,True,True,True,True,False,False,False,False,False,105.452,105.849,106.257,106.603,106.595,0.973787,0.978141,0.994041,1.01661,1.0263,True,True,True,False,False,False,False,False,True,False,104.98,105.324,105.614,105.824,105.962,0.969428,0.973281,0.988018,1.00918,1.0202,True,True,True,False,False,False,False,False,True,False,90.5512,90.7414,90.9238,91.0928,91.2569,0.836184,0.838529,0.850594,0.868696,0.87862,True,True,True,True,True,False,False,False,False,False,1.32886,-0.18155,0.0827046,0.385586,0.285032,-0.00426363,0.0150758,-0.00131822,-0.00721577,-0.0126751,0,0.0150879,-0.0139842,-0.0169476,-0.0259505,0.0160589,0.00218302,-0.00740612,-0.00693469,-0.0149386,0.017677,-0.0007

#### --- done building the pieces, now implement in one loop to cycle through all rows of df ---

In [341]:
def create_feature_cols_df(df_X_base_data,
                           n_days_features=n_days_features,
                           cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']):
    '''
    Take dataframe with date index (sorted increasing time) with multiple columns and return a new wider dataframe
    where the rows for the last n_days_features have been pivoted into additional columns
    Input:
        df_X_base_data - dataframe with date index
        n_days_features - number of prior days that are pivoted into the rows
        cols_to_normalize_to_1_for_day_0 - columns that will be normalized for day N. eg prices, volumes
    Output:
        df_X - datafram that has length of df_X_base_data.shape[0] - n_days_features and more columns than df_X_base_data
    '''
    
    df_X = pd.DataFrame()
    
    # total lenght of df
    n_data_points = df_X_base_data.shape[0]
    
    # cycle through each row of df, start at n_days_features-1 because we wouldn't have enough history for first rows
    for i in tqdm(range(n_days_features, n_data_points+1), desc='reshaping data into feature rows'):
        # i contains the rows number of df

        df_extract = df_X_base_data.iloc[i-n_days_features:i, :]

        # pull out n_days_features of rows from current position
        df_extract = df_X_base_data.iloc[i-n_days_features:i, :].copy()

        # change the index to be "days into the past" - eg current day is 0, prior day is -1, ...
        df_extract.loc[:, 'row_index'] = range(-n_days_features+1, 1)

        # make this the new index
        df_extract.set_index('row_index', inplace=True)
        
        # normalize columns from dollars to "1" - turns the columns into ratios compared to day N (index 0)
        df_extract.loc[:, cols_to_normalize_to_1_for_day_0] = df_extract.loc[:, cols_to_normalize_to_1_for_day_0] / df_extract.loc[0, cols_to_normalize_to_1_for_day_0]

        # unstack and make it tall (ie unpivot)
        df_extract = df_extract.unstack().reset_index()

        # create new column with combined field names of attribute and index
        # eg: AdjClose_-1 for the adjusted close of day N-1 or AdjHigh_-4 for the adjusted High of day N-4
        df_extract['Attribute-index'] = df_extract['Attributes'] + '_' + df_extract['row_index'].apply(str)
        # then drop Attributes and row_index columns since they are not needed anymore
        df_extract.drop(columns=['Attributes', 'row_index'], inplace=True)

        # set index one and transpose
        target_row = df_extract.set_index('Attribute-index').T
        # we now have one row of data that represents the prior n_feature_days worth of data

        # fill in the target_row index with the date from the index of the source dataframe df_X_base_data (ie, day N)
        target_row['Index'] = df_X_base_data.index.to_list()[i-1] # zero-indexed so need minus 1
        target_row = target_row.set_index('Index')

        df_X = df_X.append(target_row)
        
    return df_X

In [342]:
# split df into features and target (column setup_for_profitable_trade)
df_X_base_data = df.drop(columns=['setup_for_profitable_trade'])
df_y = df['setup_for_profitable_trade']

# add technical indicators

# RSI
df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.RSI, 14)
# add threshold columns for above 80 and below 20
df_X_base_data.loc[:, 'RSI_above_80'] = df_X_base_data.loc[:, indicator_name] > 80
df_X_base_data.loc[:, 'RSI_below_20'] = df_X_base_data.loc[:, indicator_name] < 20
# normalize to values between 0 and 1
df_X_base_data.loc[:, indicator_name] = df_X_base_data.loc[:, indicator_name] / 100

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 10)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=True)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 50)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=True)

df_X_base_data, indicator_name = add_TALib_indicator(df_X_base_data, 'AdjClose', ta.SMA, 200)
df_X_base_data = add_comparison_cols_for_indicator(df_X_base_data, 'AdjClose', indicator_name, delete_indicator_col=True)

# feature engineering: instead of dollars/absolute values, calculate change from one day to next
cols_set_vals_to_change = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose']
df_X_base_data = feat_eng_changes_values_to_change(df_X_base_data, cols_set_vals_to_change, delete_original_cols=False)

# create wide features matrix that includes prior days' data as columns
df_X = create_feature_cols_df(df_X_base_data,
                              n_days_features= 20,
                              cols_to_normalize_to_1_for_day_0 = ['AdjVolume', 'AdjOpen', 'AdjLow', 'AdjHigh', 'AdjClose'])

df_X.tail()

reshaping data into feature rows: 100%|████| 1287/1287 [01:04<00:00, 20.11it/s]


Attribute-index,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-19,AdjOpen_-18,AdjOpen_-17,AdjOpen_-16,AdjOpen_-15,AdjOpen_-14,AdjOpen_-13,AdjOpen_-12,AdjOpen_-11,AdjOpen_-10,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-19,AdjHigh_-18,AdjHigh_-17,AdjHigh_-16,AdjHigh_-15,AdjHigh_-14,AdjHigh_-13,AdjHigh_-12,AdjHigh_-11,AdjHigh_-10,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-19,AdjLow_-18,AdjLow_-17,AdjLow_-16,AdjLow_-15,AdjLow_-14,AdjLow_-13,AdjLow_-12,AdjLow_-11,AdjLow_-10,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-19,AdjClose_-18,AdjClose_-17,AdjClose_-16,AdjClose_-15,AdjClose_-14,AdjClose_-13,AdjClose_-12,AdjClose_-11,AdjClose_-10,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_-19,AdjCloseRSI14_-18,AdjCloseRSI14_-17,AdjCloseRSI14_-16,AdjCloseRSI14_-15,AdjCloseRSI14_-14,AdjCloseRSI14_-13,AdjCloseRSI14_-12,AdjCloseRSI14_-11,AdjCloseRSI14_-10,AdjCloseRSI14_-9,AdjCloseRSI14_-8,AdjCloseRSI14_-7,AdjCloseRSI14_-6,AdjCloseRSI14_-5,AdjCloseRSI14_-4,AdjCloseRSI14_-3,AdjCloseRSI14_-2,AdjCloseRSI14_-1,AdjCloseRSI14_0,RSI_above_80_-19,RSI_above_80_-18,RSI_above_80_-17,RSI_above_80_-16,RSI_above_80_-15,RSI_above_80_-14,RSI_above_80_-13,RSI_above_80_-12,RSI_above_80_-11,RSI_above_80_-10,RSI_above_80_-9,RSI_above_80_-8,RSI_above_80_-7,RSI_above_80_-6,RSI_above_80_-5,RSI_above_80_-4,RSI_above_80_-3,RSI_above_80_-2,RSI_above_80_-1,RSI_above_80_0,RSI_below_20_-19,RSI_below_20_-18,RSI_below_20_-17,RSI_below_20_-16,RSI_below_20_-15,RSI_below_20_-14,RSI_below_20_-13,RSI_below_20_-12,RSI_below_20_-11,RSI_below_20_-10,RSI_below_20_-9,RSI_below_20_-8,RSI_below_20_-7,RSI_below_20_-6,RSI_below_20_-5,RSI_below_20_-4,RSI_below_20_-3,RSI_below_20_-2,RSI_below_20_-1,RSI_below_20_0,AdjCloseSMA10_to_AdjClose_ratio_-19,AdjCloseSMA10_to_AdjClose_ratio_-18,AdjCloseSMA10_to_AdjClose_ratio_-17,AdjCloseSMA10_to_AdjClose_ratio_-16,AdjCloseSMA10_to_AdjClose_ratio_-15,AdjCloseSMA10_to_AdjClose_ratio_-14,AdjCloseSMA10_to_AdjClose_ratio_-13,AdjCloseSMA10_to_AdjClose_ratio_-12,AdjCloseSMA10_to_AdjClose_ratio_-11,AdjCloseSMA10_to_AdjClose_ratio_-10,AdjCloseSMA10_to_AdjClose_ratio_-9,AdjCloseSMA10_to_AdjClose_ratio_-8,AdjCloseSMA10_to_AdjClose_ratio_-7,AdjCloseSMA10_to_AdjClose_ratio_-6,AdjCloseSMA10_to_AdjClose_ratio_-5,AdjCloseSMA10_to_AdjClose_ratio_-4,AdjCloseSMA10_to_AdjClose_ratio_-3,AdjCloseSMA10_to_AdjClose_ratio_-2,AdjCloseSMA10_to_AdjClose_ratio_-1,AdjCloseSMA10_to_AdjClose_ratio_0,AdjClose_above_AdjCloseSMA10_-19,AdjClose_above_AdjCloseSMA10_-18,AdjClose_above_AdjCloseSMA10_-17,AdjClose_above_AdjCloseSMA10_-16,AdjClose_above_AdjCloseSMA10_-15,AdjClose_above_AdjCloseSMA10_-14,AdjClose_above_AdjCloseSMA10_-13,AdjClose_above_AdjCloseSMA10_-12,AdjClose_above_AdjCloseSMA10_-11,AdjClose_above_AdjCloseSMA10_-10,AdjClose_above_AdjCloseSMA10_-9,AdjClose_above_AdjCloseSMA10_-8,AdjClose_above_AdjCloseSMA10_-7,AdjClose_above_AdjCloseSMA10_-6,AdjClose_above_AdjCloseSMA10_-5,AdjClose_above_AdjCloseSMA10_-4,AdjClose_above_AdjCloseSMA10_-3,AdjClose_above_AdjCloseSMA10_-2,AdjClose_above_AdjCloseSMA10_-1,AdjClose_above_AdjCloseSMA10_0,AdjClose_crossed_AdjCloseSMA10_-19,AdjClose_crossed_AdjCloseSMA10_-18,AdjClose_crossed_AdjCloseSMA10_-17,AdjClose_crossed_AdjCloseSMA10_-16,AdjClose_crossed_AdjCloseSMA10_-15,AdjClose_crossed_AdjCloseSMA10_-14,AdjClose_crossed_AdjCloseSMA10_-13,AdjClose_crossed_AdjCloseSMA10_-12,AdjClose_crossed_AdjCloseSMA10_-11,AdjClose_crossed_AdjCloseSMA10_-10,AdjClose_crossed_AdjCloseSMA10_-9,AdjClose_crossed_AdjCloseSMA10_-8,AdjClose_crossed

In [343]:
df_X.shape

(1287, 440)

### Feature Engineering - Date information

In [344]:
def feat_eng_append_date_index_content(df):
    '''
    Assumes that the df index is date-time. Bolts on additional columns about the date
    '''

    df['year'] = df.index.year.values
    df['month'] = df.index.month.values
    df['week'] = df.index.week.values
    df['weekday'] = df.index.weekday.values
    df['day'] = df.index.day.values
    df['year'] = df.index.year.values
    df['year'] = df.index.year.values
    
    return df
    
df_X = feat_eng_append_date_index_content(df_X)
df_X.tail()

Attribute-index,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-19,AdjOpen_-18,AdjOpen_-17,AdjOpen_-16,AdjOpen_-15,AdjOpen_-14,AdjOpen_-13,AdjOpen_-12,AdjOpen_-11,AdjOpen_-10,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-19,AdjHigh_-18,AdjHigh_-17,AdjHigh_-16,AdjHigh_-15,AdjHigh_-14,AdjHigh_-13,AdjHigh_-12,AdjHigh_-11,AdjHigh_-10,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-19,AdjLow_-18,AdjLow_-17,AdjLow_-16,AdjLow_-15,AdjLow_-14,AdjLow_-13,AdjLow_-12,AdjLow_-11,AdjLow_-10,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-19,AdjClose_-18,AdjClose_-17,AdjClose_-16,AdjClose_-15,AdjClose_-14,AdjClose_-13,AdjClose_-12,AdjClose_-11,AdjClose_-10,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_-19,AdjCloseRSI14_-18,AdjCloseRSI14_-17,AdjCloseRSI14_-16,AdjCloseRSI14_-15,AdjCloseRSI14_-14,AdjCloseRSI14_-13,AdjCloseRSI14_-12,AdjCloseRSI14_-11,AdjCloseRSI14_-10,AdjCloseRSI14_-9,AdjCloseRSI14_-8,AdjCloseRSI14_-7,AdjCloseRSI14_-6,AdjCloseRSI14_-5,AdjCloseRSI14_-4,AdjCloseRSI14_-3,AdjCloseRSI14_-2,AdjCloseRSI14_-1,AdjCloseRSI14_0,RSI_above_80_-19,RSI_above_80_-18,RSI_above_80_-17,RSI_above_80_-16,RSI_above_80_-15,RSI_above_80_-14,RSI_above_80_-13,RSI_above_80_-12,RSI_above_80_-11,RSI_above_80_-10,RSI_above_80_-9,RSI_above_80_-8,RSI_above_80_-7,RSI_above_80_-6,RSI_above_80_-5,RSI_above_80_-4,RSI_above_80_-3,RSI_above_80_-2,RSI_above_80_-1,RSI_above_80_0,RSI_below_20_-19,RSI_below_20_-18,RSI_below_20_-17,RSI_below_20_-16,RSI_below_20_-15,RSI_below_20_-14,RSI_below_20_-13,RSI_below_20_-12,RSI_below_20_-11,RSI_below_20_-10,RSI_below_20_-9,RSI_below_20_-8,RSI_below_20_-7,RSI_below_20_-6,RSI_below_20_-5,RSI_below_20_-4,RSI_below_20_-3,RSI_below_20_-2,RSI_below_20_-1,RSI_below_20_0,AdjCloseSMA10_to_AdjClose_ratio_-19,AdjCloseSMA10_to_AdjClose_ratio_-18,AdjCloseSMA10_to_AdjClose_ratio_-17,AdjCloseSMA10_to_AdjClose_ratio_-16,AdjCloseSMA10_to_AdjClose_ratio_-15,AdjCloseSMA10_to_AdjClose_ratio_-14,AdjCloseSMA10_to_AdjClose_ratio_-13,AdjCloseSMA10_to_AdjClose_ratio_-12,AdjCloseSMA10_to_AdjClose_ratio_-11,AdjCloseSMA10_to_AdjClose_ratio_-10,AdjCloseSMA10_to_AdjClose_ratio_-9,AdjCloseSMA10_to_AdjClose_ratio_-8,AdjCloseSMA10_to_AdjClose_ratio_-7,AdjCloseSMA10_to_AdjClose_ratio_-6,AdjCloseSMA10_to_AdjClose_ratio_-5,AdjCloseSMA10_to_AdjClose_ratio_-4,AdjCloseSMA10_to_AdjClose_ratio_-3,AdjCloseSMA10_to_AdjClose_ratio_-2,AdjCloseSMA10_to_AdjClose_ratio_-1,AdjCloseSMA10_to_AdjClose_ratio_0,AdjClose_above_AdjCloseSMA10_-19,AdjClose_above_AdjCloseSMA10_-18,AdjClose_above_AdjCloseSMA10_-17,AdjClose_above_AdjCloseSMA10_-16,AdjClose_above_AdjCloseSMA10_-15,AdjClose_above_AdjCloseSMA10_-14,AdjClose_above_AdjCloseSMA10_-13,AdjClose_above_AdjCloseSMA10_-12,AdjClose_above_AdjCloseSMA10_-11,AdjClose_above_AdjCloseSMA10_-10,AdjClose_above_AdjCloseSMA10_-9,AdjClose_above_AdjCloseSMA10_-8,AdjClose_above_AdjCloseSMA10_-7,AdjClose_above_AdjCloseSMA10_-6,AdjClose_above_AdjCloseSMA10_-5,AdjClose_above_AdjCloseSMA10_-4,AdjClose_above_AdjCloseSMA10_-3,AdjClose_above_AdjCloseSMA10_-2,AdjClose_above_AdjCloseSMA10_-1,AdjClose_above_AdjCloseSMA10_0,AdjClose_crossed_AdjCloseSMA10_-19,AdjClose_crossed_AdjCloseSMA10_-18,AdjClose_crossed_AdjCloseSMA10_-17,AdjClose_crossed_AdjCloseSMA10_-16,AdjClose_crossed_AdjCloseSMA10_-15,AdjClose_crossed_AdjCloseSMA10_-14,AdjClose_crossed_AdjCloseSMA10_-13,AdjClose_crossed_AdjCloseSMA10_-12,AdjClose_crossed_AdjCloseSMA10_-11,AdjClose_crossed_AdjCloseSMA10_-10,AdjClose_crossed_AdjCloseSMA10_-9,AdjClose_crossed_AdjCloseSMA10_-8,AdjClose_crossed

##### just testing aroung: can also use only one or a few columns and then join the dataframes back together

In [345]:
#df_XO = create_feature_cols_df(df_X_base_data[['AdjOpen']])
#df_XO.tail()

In [346]:
#df_X.join(df_XO).tail()

## Machine Learning

### Build Model

In [347]:
# need to remove first n_days_features rows - they were eliminated during build of df_X and sizes need to match
df_y = df_y.iloc[n_days_features-1:]

In [348]:
# remove all rows that have any NaNs in them - they come from technical indicators or the reshaping and we just don't have 
# any good strategy for imputation other than starting with more time series data
df_X = df_X.loc[df_X.notnull().all(axis=1), :]
df_X.shape

(1088, 445)

In [349]:
# now we need to keep the same rows that we kept in df_X in df_y. We'll use the index to filter
df_y = df_y.loc[df_X.index]
df_y.shape

(1088,)

In [350]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, shuffle=False)

In [351]:
pipeline = Pipeline([
    ('standardScaler', StandardScaler()),
    ('randomForest', RandomForestClassifier())
])
pipeline.get_params()

{'memory': None,
 'steps': [('standardScaler',
   StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('randomForest',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=False, random_state=None, verbose=0,
               warm_start=False))],
 'standardScaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'randomForest': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators='war

In [352]:
pipeline.fit(X_train, y_train)

C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('standardScaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [353]:
y_pred = pipeline.predict(X_test)

C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [354]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.73      0.87      0.79       156
        True       0.35      0.18      0.24        62

   micro avg       0.67      0.67      0.67       218
   macro avg       0.54      0.52      0.51       218
weighted avg       0.62      0.67      0.63       218



In [355]:
accuracy_score(y_test, y_pred)

0.6743119266055045

In [358]:
confusion_matrix(y_test, y_pred, labels=[False, True])

array([[136,  20],
       [ 51,  11]], dtype=int64)

In [357]:
importances = pipeline.named_steps['randomForest'].feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print(str(f + 1) + ". feature " + X_train.columns[indices[f]] + " importance: " + str(importances[indices[f]]))

Feature ranking:
1. feature week importance: 0.016858011200165383
2. feature AdjCloseSMA200_to_AdjClose_ratio_-2 importance: 0.015775506780390837
3. feature AdjCloseSMA50_to_AdjClose_ratio_-6 importance: 0.013981640331008663
4. feature AdjCloseSMA200_to_AdjClose_ratio_-3 importance: 0.012788884333931
5. feature AdjCloseSMA200_to_AdjClose_ratio_-1 importance: 0.01059688385392386
6. feature AdjCloseRSI14_-19 importance: 0.010457610020236778
7. feature AdjOpen_-16 importance: 0.010085128592833257
8. feature AdjCloseSMA200_to_AdjClose_ratio_-5 importance: 0.009970815518350901
9. feature AdjCloseSMA50_to_AdjClose_ratio_-15 importance: 0.009966877659524689
10. feature AdjCloseSMA200_to_AdjClose_ratio_-6 importance: 0.009708938030205231
11. feature AdjCloseSMA200_to_AdjClose_ratio_-16 importance: 0.009492809042783688
12. feature AdjCloseSMA10_to_AdjClose_ratio_-18 importance: 0.009190940538327917
13. feature AdjCloseSMA50_to_AdjClose_ratio_-17 importance: 0.008990232994124192
14. feature AdjV

125. feature AdjVolume_-8 importance: 0.0031780091932457496
126. feature AdjOpen_chg_-2 importance: 0.003163285824623922
127. feature AdjVolume_chg_-5 importance: 0.0031588070599639405
128. feature AdjCloseSMA50_to_AdjClose_ratio_-4 importance: 0.003134102204151052
129. feature AdjHigh_chg_-16 importance: 0.0031191590181138527
130. feature AdjCloseSMA200_to_AdjClose_ratio_-4 importance: 0.003117311876154765
131. feature AdjOpen_-10 importance: 0.0030558531001132505
132. feature AdjClose_chg_-6 importance: 0.0030415028995958437
133. feature AdjClose_-15 importance: 0.0030224153808317035
134. feature AdjCloseSMA10_to_AdjClose_ratio_-14 importance: 0.002990072238143171
135. feature AdjVolume_-11 importance: 0.0029833897634338985
136. feature AdjHigh_-16 importance: 0.002951940464176247
137. feature AdjLow_chg_-8 importance: 0.0029294734260985485
138. feature AdjClose_chg_-10 importance: 0.00292815521145154
139. feature AdjCloseSMA10_to_AdjClose_ratio_-5 importance: 0.0028017301058767957
1

248. feature AdjClose_crossed_AdjCloseSMA50_-17 importance: 0.0011167539334740826
249. feature AdjVolume_chg_-3 importance: 0.0011156858535324408
250. feature AdjLow_chg_-11 importance: 0.0011072248048155432
251. feature AdjHigh_chg_-12 importance: 0.0010643474653590477
252. feature AdjLow_chg_-12 importance: 0.0010460892329333562
253. feature AdjVolume_-6 importance: 0.001045045045045045
254. feature AdjClose_above_AdjCloseSMA50_-14 importance: 0.0010249049786629828
255. feature AdjClose_chg_-18 importance: 0.0009912390471940216
256. feature AdjCloseRSI14_-4 importance: 0.0009639179642854298
257. feature AdjClose_crossed_AdjCloseSMA10_-8 importance: 0.0009615177638759067
258. feature AdjClose_chg_-5 importance: 0.0008987603305785124
259. feature AdjOpen_-7 importance: 0.0008392656725990062
260. feature AdjClose_-18 importance: 0.0008339928587204591
261. feature AdjClose_-13 importance: 0.0008088842975206613
262. feature AdjLow_-10 importance: 0.0007978383689511176
263. feature AdjHigh

379. feature RSI_above_80_-13 importance: 0.0
380. feature RSI_below_20_-5 importance: 0.0
381. feature RSI_above_80_-14 importance: 0.0
382. feature RSI_above_80_-15 importance: 0.0
383. feature RSI_above_80_-16 importance: 0.0
384. feature RSI_above_80_-17 importance: 0.0
385. feature RSI_above_80_-18 importance: 0.0
386. feature RSI_above_80_-19 importance: 0.0
387. feature AdjClose_above_AdjCloseSMA50_-19 importance: 0.0
388. feature RSI_above_80_-4 importance: 0.0
389. feature RSI_above_80_-3 importance: 0.0
390. feature RSI_above_80_-2 importance: 0.0
391. feature RSI_below_20_-3 importance: 0.0
392. feature RSI_below_20_-6 importance: 0.0
393. feature RSI_below_20_-7 importance: 0.0
394. feature RSI_below_20_-8 importance: 0.0
395. feature RSI_below_20_-9 importance: 0.0
396. feature RSI_below_20_-10 importance: 0.0
397. feature RSI_below_20_-11 importance: 0.0
398. feature RSI_below_20_-12 importance: 0.0
399. feature RSI_below_20_-13 importance: 0.0
400. feature RSI_below_20_-

In [284]:
pipeline.named_steps['randomForest'].n_features_

445

In [270]:
y_pred.shape

(220,)

In [271]:
y_pred

array([False,  True,  True, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False,  True,  True, False,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False,  True, False, False, False,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False,  True, False, False,  True,  True, False, False, False,
       False, False,

### Improve RandomForest model

In [246]:
parameters = {
    'randomForest__min_samples_leaf': [1, 2, 5, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, scoring='f1', n_jobs=-1, verbose=6)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:    3.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    4.0s finished
C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  r

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('standardScaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'randomForest__min_samples_leaf': [1, 2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=6)

In [247]:
cv.best_params_

{'randomForest__min_samples_leaf': 5}

In [248]:
y_pred = cv.predict(X_test)

C:\Users\Lindstaedt\Anaconda3\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [249]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.64      0.80      0.71       146
        True       0.22      0.11      0.14        74

   micro avg       0.57      0.57      0.57       220
   macro avg       0.43      0.45      0.43       220
weighted avg       0.50      0.57      0.52       220



In [250]:
accuracy_score(y_test, y_pred)

0.5681818181818182

In [251]:
confusion_matrix(y_test, y_pred, labels=[False, True])

array([[117,  29],
       [ 66,   8]], dtype=int64)

In [221]:
y_pred

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False,  True,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True,

In [222]:
X_test.join(y_test)

,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_to_AdjClose_ratio_-9,AdjCloseRSI14_to_AdjClose_ratio_-8,AdjCloseRSI14_to_AdjClose_ratio_-7,AdjCloseRSI14_to_AdjClose_ratio_-6,AdjCloseRSI14_to_AdjClose_ratio_-5,AdjCloseRSI14_to_AdjClose_ratio_-4,AdjCloseRSI14_to_AdjClose_ratio_-3,AdjCloseRSI14_to_AdjClose_ratio_-2,AdjCloseRSI14_to_AdjClose_ratio_-1,AdjCloseRSI14_to_AdjClose_ratio_0,AdjClose_above_AdjCloseRSI14_-9,AdjClose_above_AdjCloseRSI14_-8,AdjClose_above_AdjCloseRSI14_-7,AdjClose_above_AdjCloseRSI14_-6,AdjClose_above_AdjCloseRSI14_-5,AdjClose_above_AdjCloseRSI14_-4,AdjClose_above_AdjCloseRSI14_-3,AdjClose_above_AdjCloseRSI14_-2,AdjClose_above_AdjCloseRSI14_-1,AdjClose_above_AdjCloseRSI14_0,AdjClose_crossed_AdjCloseRSI14_-9,AdjClose_crossed_AdjCloseRSI14_-8,AdjClose_crossed_AdjCloseRSI14_-7,AdjClose_crossed_AdjCloseRSI14_-6,AdjClose_crossed_AdjCloseRSI14_-5,...,AdjClose_above_AdjCloseSMA200_-8,AdjClose_above_AdjCloseSMA200_-7,AdjClose_above_AdjCloseSMA200_-6,AdjClose_above_AdjCloseSMA200_-5,AdjClose_above_AdjCloseSMA200_-4,AdjClose_above_AdjCloseSMA200_-3,AdjClose_above_AdjCloseSMA200_-2,AdjClose_above_AdjCloseSMA200_-1,AdjClose_above_AdjCloseSMA200_0,AdjClose_crossed_AdjCloseSMA200_-9,AdjClose_crossed_AdjCloseSMA200_-8,AdjClose_crossed_AdjCloseSMA200_-7,AdjClose_crossed_AdjCloseSMA200_-6,AdjClose_crossed_AdjCloseSMA200_-5,AdjClose_crossed_AdjCloseSMA200_-4,AdjClose_crossed_AdjCloseSMA200_-3,AdjClose_crossed_AdjCloseSMA200_-2,AdjClose_crossed_AdjCloseSMA200_-1,AdjClose_crossed_AdjCloseSMA200_0,AdjVolume_chg_-9,AdjVolume_chg_-8,AdjVolume_chg_-7,AdjVolume_chg_-6,AdjVolume_chg_-5,AdjVolume_chg_-4,AdjVolume_chg_-3,AdjVolume_chg_-2,AdjVolume_chg_-1,AdjVolume_chg_0,AdjOpen_chg_-9,AdjOpen_chg_-8,AdjOpen_chg_-7,AdjOpen_chg_-6,AdjOpen_chg_-5,AdjOpen_chg_-4,AdjOpen_chg_-3,AdjOpen_chg_-2,AdjOpen_chg_-1,AdjOpen_chg_0,AdjLow_chg_-9,AdjLow_chg_-8,AdjLow_chg_-7,AdjLow_chg_-6,AdjLow_chg_-5,AdjLow_chg_-4,AdjLow_chg_-3,AdjLow_chg_-2,AdjLow_chg_-1,AdjLow_chg_0,AdjHigh_chg_-9,AdjHigh_chg_-8,AdjHigh_chg_-7,AdjHigh_chg_-6,AdjHigh_chg_-5,AdjHigh_chg_-4,AdjHigh_chg_-3,AdjHigh_chg_-2,AdjHigh_chg_-1,AdjHigh_chg_0,AdjClose_chg_-9,AdjClose_chg_-8,AdjClose_chg_-7,AdjClose_chg_-6,AdjClose_chg_-5,AdjClose_chg_-4,AdjClose_chg_-3,AdjClose_chg_-2,AdjClose_chg_-1,AdjClose_chg_0,year,month,week,weekday,day,setup_for_profitable_trade
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-05-04,0.744164,0.735385,0.782639,0.842694,0.599294,0.869886,1.41,1.70791,1.93948,1,0.972154,0.979388,0.982187,0.986009,0.982272,0.983415,0.990308,1.00696,0.993653,1,0.969689,0.97832,0.984776,0.982738,0.979747,0.980699,1.00041,1.00646,1.00238,1,0.972841,0.981963,0.986695,0.983308,0.982854,0.98258,0.99417,1.00706,0.989438,1,0.970927,0.980277,0.986419,0.980351,0.981301,0.980345,1.00048,1.00669,1.00362,1,0.393203,0.430103,0.450191,0.415695,0.419204,0.413484,0.475499,0.488675,0.47101,0.450385,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,-0.257236,-0.0117967,0.0642577,0.0767336,-0.288836,0.451517,0.620905,0.211279,0.135592,-0.484399,0.008639,0.00744173,0.00285714,0.00389132,-0.00378971,0.00116382,0.00700951,0.016816,-0.0132168,0.0063878,0.00488807,0.0093761,0.00481911,-0.00343227,-0.000461722,-0.000279115,0.0117959,

## next: add backtesting

## use TPOT
## add column for stock
## add additional technical indicators
## add crossover metrics